In [156]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pymysql

In [157]:
conn = pymysql.connect(
    host = 'localhost',
    user='root',
    password='apmsetup',
    db='WebData',
    charset='utf8'
)

In [169]:
cur = conn.cursor()
cur.execute('select * from data')
print(cur.fetchall())



((1, '전북銀, <b>대구</b>에 ‘영남오토금융센터’ 개소', 'http://www.idaegu.co.kr/news.php?code=ec04&mode=view&num=251295', '이에 따라 <b>대구</b>·경북을 주력 영업기반으로 하는 DGB금융그룹 계열사인 <b>대구은행</b> 및 DGB캐피탈과의 치열한 자동차 할부금융 영업경쟁이 예상된다. 현재 <b>대구</b>에 점포를 두고 있는 역외 지방금융그룹은 BNK금융그룹의... ', '2018-06-06 21:18:00'), (2, '전북銀, <b>대구</b>에 ‘영남오토금융센터’ 개소', 'http://www.idaegu.co.kr/news.php?code=ec04&mode=view&num=251295', '이에 따라 <b>대구</b>·경북을 주력 영업기반으로 하는 DGB금융그룹 계열사인 <b>대구은행</b> 및 DGB캐피탈과의 치열한 자동차 할부금융 영업경쟁이 예상된다. 현재 <b>대구</b>에 점포를 두고 있는 역외 지방금융그룹은 BNK금융그룹의... ', '2018-06-06 21:18:00'), (3, 'DGB 금융지주·<b>대구은행</b>, 1분기 자본비율 모두 개선', 'http://www.idaegu.co.kr/news.php?code=ec01&mode=view&num=251230', 'DGB금융지주와 <b>대구은행</b>의 올 1분기(3월말) BIS(국제결제<b>은행</b>) 기준 자본비율이 전분기 대비 모두 개선된 것으로 나타났다. BIS 자본비율은 위험자산(부실채권) 대비 자기자본비율로 <b>은행</b>의 안전성과... ', '2018-06-06 21:04:00'), (4, 'DGB 스마트뱅킹 이용자 대상 월드컵 기념 선물 증정 이벤트', 'http://www.idaegu.com?c=5&uid=388854', 'DGB 스마트뱅킹 이용자 대상 월드컵 기념 선물 증정 이벤트 DGB 스마트뱅킹 이용자 대상 월드컵 기념 선물 증정 이벤트 DGB<b>대구은행</b>은 러시아월드컵에 참가하는

In [159]:
#[CODE 1]
def get_request_url(url):
    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", "id")
    req.add_header("X-Naver-Client-Secret", "")

    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None


In [160]:
#[CODE 2]
def getNaverSearchResult(sNode, search_text, page_start, display):
    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % sNode
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(search_text), page_start, display)
    #parameters = "?query=%s" % urllib.parse.quote(search_text)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [161]:
#[CODE 3]
def getPostData(post, jsonResult):
    
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    #Tue, 14 Feb 2017 18:46:00 +0900
    pDate = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    jsonResult.append({'title':title, 'description': description,
                    'org_link':org_link, 'link': org_link,
                    'pDate':pDate})
    return    

In [165]:
def main():

    jsonResult = []

    # 'news', 'blog', 'cafearticle'
    sNode = 'news'
    search_text = '대구은행'
    display_count = 10
    
    jsonSearch = getNaverSearchResult(sNode, search_text, 1, display_count)
    
    if ((jsonSearch != None) and (jsonSearch['display'] != 0)):
        for post in jsonSearch['items']:
            getPostData(post, jsonResult)
        
    # nStart = jsonSearch['start'] + jsonSearch['display']
    # jsonSearch = getNaverSearchResult(sNode, search_text, nStart, display_count)
    
#     with open('%s_naver_%s.json' % (search_text, sNode), 'w', encoding='utf8') as outfile:
#         retJson = json.dumps(jsonResult,
#                         indent=4, sort_keys=True,
#                         ensure_ascii=False)
#         outfile.write(retJson)

    print(len(jsonResult))
    
    for i in range(len(jsonResult)):
        try :
            sql = "INSERT INTO data(id,title,link,des,date) VALUES (null ,'"+jsonResult[i]['title']+"','"+jsonResult[i]['link']+"','"+jsonResult[i]['description']+"','"+jsonResult[i]['pDate']+"')"
            cur.execute(sql)
            conn.commit()
        except BaseException :
            continue
        

In [166]:
main()

[2018-06-06 21:24:29.278089] Url Request Success
10
